In [ ]:
import requests
import json
import os
import shutil
import pwd
import grp

# Ades2 Deploy and Undeploy Demo

In [ ]:
services_folder="/usr/lib/cgi-bin"

## List Processes
Provides a list of all processes

In [ ]:
response = requests.get('http://localhost/ogc-api/processes')
response.json()

## Inspect deployed services
In the Ades container/pod inspect the services folder

In [ ]:
for path, subdirs, files in os.walk(services_folder):
    for name in files:
        print(os.path.join(path, name))

## Deploy the service using the application package
The body of the POST request will include the application package.</br>
The response returns 201 CREATED to indicate that the service has been successfully deployed.</br>
The response Location header provides the path to the service details.

In [ ]:
url = "http://localhost/ogc-api/processes/"
payload = open('data/dnbr_app/dnbr.cwl', 'rb').read()

headers = {'Content-Type': 'application/cwl'}
response = requests.request("POST", url, headers=headers, data=payload)

print(f"Body: {response.json()}\n")
print(f"Headers: {response.headers}\n")
print(f"Status code: {response.status_code}")

## Inspect the application service's files

In [ ]:
for path, subdirs, files in os.walk("/usr/lib/cgi-bin/"):
    for name in files:
        if "dnbr" in name or "dnbr" in path:
            print(os.path.join(path, name))

## Execute the deployed service
Request body is json as defined by API Processes to define the inputs and outputs, consistent with the CWL Workflow application package.</br>
The response returns 201 CREATED to indicate that the job has been successfully initiated.</br>
The response Location header provides the path (/processes/{application_name}/jobs/{job_id}) to follow the job status.

In [ ]:
url = "http://localhost/ogc-api/processes/dnbr"

payload = open('data/dnbr_app/dnbr_execute_payload.json', 'rb').read()
headers = {
  'Accept': 'application/json',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(f"Headers: {response.headers}\n")
print(f"Status code: {response.status_code}")

In [ ]:
location = response.headers["Location"]
print(location)

## Get Job Status
Check the status of a previously submitted job - using the URL returned in the Location header of the execute request.</br>
The response body json provides a status string (success/running/failed) and a % progress..</br>
In the case of a failure then a descriptive message is provided.

In [ ]:
get_status_response = requests.get(location)
get_status_response.json()

## Get Job Result
Returns details of the outputs for a successful job execution.</br>
The response body provides json data that includes the reference to the STAC file that indexes the processing outputs.

In [ ]:
get_result_url = f"{location}/results"
get_result_response = requests.request("GET", get_result_url)
get_result_response_json=json.loads(get_result_response.text)
print(json.dumps(get_result_response_json,indent=2))

## Inspect Job Result

In [ ]:
result_url = get_result_response_json["stac"]["href"]
print(result_url)
result_response = requests.request("GET", result_url)
print(result_response.text)

## Undeploy the service
Undeploys the service

In [ ]:
url = "http://localhost/ogc-api/processes/dnbr"
response = requests.request("DELETE", url)
print(response.headers)

In [ ]:
for path, subdirs, files in os.walk(services_folder):
    for name in files:
        print(os.path.join(path, name))

# Deploying on namespace

## Creating the namespace
Creating the folder for the namespace

In [ ]:
default_services_path="/usr/lib/cgi-bin"
my_namespace_path="/opt/zooservices_namespaces/eoepca"

In [ ]:
if not os.path.exists(my_namespace_path):
    os.makedirs(my_namespace_path) 

Copying the Deploy and Undeploy services in the namespace folder

In [ ]:
shutil.copyfile(f"{default_services_path}/DeployProcess.py", f"{my_namespace_path}/DeployProcess.py")
shutil.copyfile(f"{default_services_path}/DeployProcess.zcfg", f"{my_namespace_path}/DeployProcess.zcfg")
shutil.copyfile(f"{default_services_path}/UndeployProcess.py", f"{my_namespace_path}/UndeployProcess.py")
shutil.copyfile(f"{default_services_path}/UndeployProcess.zcfg", f"{my_namespace_path}/UndeployProcess.zcfg")

Granting permissions to the namespace folder

In [ ]:
user_id=pwd.getpwnam("www-data")[2]
group_id=grp.getgrnam('www-data')[2]

In [ ]:
for dirpath, dirnames, filenames in os.walk(my_namespace_path):
    shutil.chown(dirpath, user_id, group_id)
    for filename in filenames:
        shutil.chown(os.path.join(dirpath, filename), user_id,group_id)

## Inspect namespace services
Provides a list of all services in the namespace *eoepca*

In [ ]:
response = requests.get('http://localhost/eoepca/ogc-api/processes')
response.json()

## Deploying application package

In [ ]:
url = "http://localhost/eoepca/ogc-api/processes/"
payload = open('data/dnbr_app/dnbr.cwl', 'rb').read()

headers = {'Content-Type': 'application/cwl'}
response = requests.request("POST", url, headers=headers, data=payload)

print(f"Body: {response.json()}\n")
print(f"Headers: {response.headers}")
print(f"Status code: {response.status_code}")

In [ ]:
response = requests.get('http://localhost/eoepca/ogc-api/processes/dnbr')
response.json()

## Execute the application

In [ ]:
url = "http://localhost/eoepca/ogc-api/processes/dnbr"

payload = open('data/dnbr_app/dnbr_execute_payload.json', 'rb').read()
headers = {
  'Accept': 'application/json',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(f"Headers: {response.headers}\n")
print(f"Status code: {response.status_code}")

In [ ]:
location = response.headers["Location"]
print(location)

## Get Job Status

In [ ]:
get_status_response = requests.get(location)
get_status_response.json()

## Get Job Result

In [ ]:
get_result_url = f"{location}/results"
get_result_response = requests.request("GET", get_result_url)
get_result_response_json=json.loads(get_result_response.text)
print(json.dumps(get_result_response_json,indent=2))

## Inspect Job Result

In [ ]:
result_url = get_result_response_json["stac"]["href"]
print(result_url)
result_response = requests.request("GET", result_url)
print(result_response.text)

## Undeploy the service

In [ ]:
url = "http://localhost/eoepca/ogc-api/processes/dnbr"
response = requests.request("DELETE", url)
print(response.headers)

In [ ]:
for path, subdirs, files in os.walk(my_namespace_path):
    for name in files:
        print(os.path.join(path, name))